In [2]:
import torch
import torchvision.models as models

# Initialize SqueezeNet model with pretrained ImageNet weights
model = models.squeezenet1_1(pretrained=True)  # Use squeezenet1_1 (recommended)

# Set to evaluation mode (important for inference)
model.eval()

# Print the model architecture (optional)
print(model)

SqueezeNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
    (3): Fire(
      (squeeze): Conv2d(64, 16, kernel_size=(1, 1), stride=(1, 1))
      (squeeze_activation): ReLU(inplace=True)
      (expand1x1): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1))
      (expand1x1_activation): ReLU(inplace=True)
      (expand3x3): Conv2d(16, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (expand3x3_activation): ReLU(inplace=True)
    )
    (4): Fire(
      (squeeze): Conv2d(128, 16, kernel_size=(1, 1), stride=(1, 1))
      (squeeze_activation): ReLU(inplace=True)
      (expand1x1): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1))
      (expand1x1_activation): ReLU(inplace=True)
      (expand3x3): Conv2d(16, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (expand3x3_activation): ReLU(inplace=True)
    )
    (5): MaxPool2d

In [3]:
from torchinfo import summary

input_size = (1, 3, 224, 224)  # (batch_size, channels, height, width)
summary(model, input_size=input_size)

Layer (type:depth-idx)                   Output Shape              Param #
SqueezeNet                               [1, 1000]                 --
├─Sequential: 1-1                        [1, 512, 13, 13]          --
│    └─Conv2d: 2-1                       [1, 64, 111, 111]         1,792
│    └─ReLU: 2-2                         [1, 64, 111, 111]         --
│    └─MaxPool2d: 2-3                    [1, 64, 55, 55]           --
│    └─Fire: 2-4                         [1, 128, 55, 55]          --
│    │    └─Conv2d: 3-1                  [1, 16, 55, 55]           1,040
│    │    └─ReLU: 3-2                    [1, 16, 55, 55]           --
│    │    └─Conv2d: 3-3                  [1, 64, 55, 55]           1,088
│    │    └─ReLU: 3-4                    [1, 64, 55, 55]           --
│    │    └─Conv2d: 3-5                  [1, 64, 55, 55]           9,280
│    │    └─ReLU: 3-6                    [1, 64, 55, 55]           --
│    └─Fire: 2-5                         [1, 128, 55, 55]          --
│  

In [5]:
import torch
from torchvision import models, transforms
from PIL import Image

# 1. Load pre-trained SqueezeNet1_1 model
model = models.squeezenet1_1(weights='IMAGENET1K_V1')
model.eval()  # Set to evaluation mode

# 2. Preprocess your image
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],   # Standard ImageNet means
        std=[0.229, 0.224, 0.225],    # Standard ImageNet stds
    ),
])

img_path = "pexels-photo-1108099.jpg"  # Replace with your image file
input_image = Image.open(img_path).convert('RGB')
input_tensor = preprocess(input_image)
input_batch = input_tensor.unsqueeze(0)  # Add batch dimension

# 3. Run inference
with torch.no_grad():
    output = model(input_batch)

# 4. Get prediction
probabilities = torch.nn.functional.softmax(output[0], dim=0)
top5_prob, top5_catid = torch.topk(probabilities, 5)

# Download label list if you don't have it
import requests
labels = requests.get("https://raw.githubusercontent.com/pytorch/hub/master/imagenet_classes.txt").text.splitlines()

print("Top-5 Predictions:")
for i in range(top5_prob.size(0)):
    print(f"{labels[top5_catid[i]]}: {top5_prob[i].item()*100:.2f}%")

Top-5 Predictions:
golden retriever: 77.95%
clumber: 8.25%
Pekinese: 5.75%
Labrador retriever: 3.82%
Sussex spaniel: 1.76%
